나만의 모델로 구성해보자

In [0]:
import torch
import torchvision 

In [113]:
trans = torchvision.transforms.Compose([
                                              torchvision.transforms.ToTensor()
])

#data 가져오기
train_dataset = torchvision.datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, transform=trans)
val_dataset = torchvision.datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, transform=trans)
test_dataset = torchvision.datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=False, transform=trans)

indices = list(range(len(train_dataset.train_data)))
split = int(len(indices)*0.2)
train_idx, val_idx = indices[:split], indices[split:]

train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32, sampler=train_sampler)
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_idx)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=32, sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32)


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [0]:
from torch import nn

Fashion data는 28*28 grayscale 이미지를 사용하기 때문에
Convolutional layer를 사용하는 것이 더 좋은 성능을 낼 것이라고 생각함.

모델 구성

1. conv
2. maxpool
3. conv
4. maxpool
5. conv
6. fc
7. fc


In [0]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 128, 3)
        self.conv2 = nn.Conv2d(128, 256, 3, stride=2)
        self.conv3 = nn.Conv2d(256, 64, 3)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64,128)
        self.fc2 = nn.Linear(128,10)

    def forward(self, x):
        x = self.maxpool(nn.functional.relu(self.conv1(x)))
        x = self.maxpool(nn.functional.relu(self.conv2(x)))
        x = nn.functional.relu(self.conv3(x))
        x = x.view(x.size()[0],-1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.fc2(x))

        return x

Cost 함수 정의

분류문제에서는 Cross entropy가 주로 loss로 사용된다.

In [0]:
model = Net()
criterion = nn.CrossEntropyLoss()

Optimizer 정의

Optimizer로 Adam 정의해보자. learning Rate는 0.03으로 했을 때 잘 안됐음.
0.001로 하니까 잘 줄어든다.

In [0]:
optimizer = torch.optim.Adam(model.parameters())

In [118]:
MAX_EPOCH = 2
for epoch in range(MAX_EPOCH):
    total_loss = 0
    step = 0
    for X, y in train_loader:
        step+=1
        optimizer.zero_grad()   #gradient 초기화
        output = model(X)
        loss = criterion(output,y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if step%20==0:
            print('{}, {} : {}'.format(epoch,step, total_loss/20))
            total_loss = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0, 20 : 2.2683673620224
0, 40 : 2.081495761871338
0, 60 : 2.0335630595684053
0, 80 : 1.9605980217456818
0, 100 : 1.9058535933494567
0, 120 : 1.8196874678134918
0, 140 : 1.8183358430862426
0, 160 : 1.7901222467422486
0, 180 : 1.7707946658134461
0, 200 : 1.8061235666275024
0, 220 : 1.789924931526184
0, 240 : 1.7750638782978059
0, 260 : 1.7964950144290923
0, 280 : 1.7709680676460267
0, 300 : 1.7830153822898864
0, 320 : 1.728974860906601
0, 340 : 1.7531466901302337
0, 360 : 1.7582011640071868
1, 20 : 1.7360978543758392
1, 40 : 1.7414471328258514
1, 60 : 1.728420662879944
1, 80 : 1.7788620829582213
1, 100 : 1.7772013604640962
1, 120 : 1.7206775963306427
1, 140 : 1.7410360991954803
1, 160 : 1.7523872137069703
1, 180 : 1.7744685411453247
1, 200 : 1.7345878958702088
1, 220 : 1.7341416835784913
1, 240 : 1.7417162239551545
1, 260 : 1.7448496520519257
1, 280 : 1.7343193471431733
1, 300 : 1.7391379296779632
1, 320 : 1.7405280470848083
1, 340 : 1.7110448002815246
1, 360 : 1.7003327012062073


In [123]:
total_loss = 0
step = 0
total = 0
correct = 0
with torch.no_grad():
    for val_x, val_y in val_loader:
        step+=1
        val_output = model(val_x)
        v_loss = criterion(val_output, val_y)
        total_loss += v_loss.item()
        _, predicted = torch.max(val_output.data, 1)
        total += val_y.size(0)
        correct += (predicted == val_y).sum().item()
        if step%100 == 0:
            print('{} {} : {}'.format(epoch, step, correct/total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


1 100 : 0.7628125
1 200 : 0.75640625
1 300 : 0.7530208333333334
1 400 : 0.753515625
1 500 : 0.7540625
1 600 : 0.7548958333333333
1 700 : 0.7537946428571428
1 800 : 0.753828125
1 900 : 0.7539583333333333
1 1000 : 0.75415625
1 1100 : 0.7541761363636363
1 1200 : 0.753671875
1 1300 : 0.7540384615384615
1 1400 : 0.75375
1 1500 : 0.7533958333333334
